In [7]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

# File url
file_url = '../datasets/foxnewshealth.txt'
dest_url = './run_data/foxnews_scrapt_v2.csv'

# Basic list
csv_header = ['tweetid','date','title']
frame_header = ['tweetid','date','title','what']

# Support scripts
%run -i scripts.py

In [3]:
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame.head()

,tweetid,date,title
0,585942799561928704,Wed Apr 08 23:10:24 +0000 2015,Injury prevention programs unpopular with high...
1,585934004089683969,Wed Apr 08 22:35:27 +0000 2015,6 dietary changes to make midlife http://ow.ly...
2,585926494054195201,Wed Apr 08 22:05:36 +0000 2015,Massachusetts governor gets head shaved to sup...
3,585921515922788354,Wed Apr 08 21:45:49 +0000 2015,Dad wins 3 marathons in 8 days; winnings to he...
4,585915150449934338,Wed Apr 08 21:20:32 +0000 2015,Possible cure for melanoma? http://ow.ly/LlLg8


In [4]:
frame.shape

(2000, 3)

In [5]:
frame.loc[:,'title'][0]

'Injury prevention programs unpopular with high school coaches http://ow.ly/Lma9z'

In [5]:
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
frame['title'] = frame['title'].map(lambda row: split_txt_form_url(regex_url, row))
frame.head()

http://ow.ly/Lma9z
http://ow.ly/LlYfl
http://ow.ly/LlLI3
http://ow.ly/LlLyZ
http://ow.ly/LlLg8
http://ow.ly/LlL69
http://ow.ly/LlKYU
http://ow.ly/LlmOO
http://ow.ly/LlmKx
http://ow.ly/Llmaj
http://ow.ly/LllIf
http://ow.ly/LllF0
http://ow.ly/LllBM
http://ow.ly/Lllze
http://ow.ly/LllvX
http://ow.ly/LlRmW
http://ow.ly/LllsI
http://ow.ly/LllpI
http://ow.ly/LlllW
http://ow.ly/LllhY
http://ow.ly/Lllah
http://ow.ly/LlkWI
http://ow.ly/Llc4n
http://ow.ly/Ljqd1
http://ow.ly/LjeGL
http://ow.ly/LiUWe
http://ow.ly/LiUSJ
http://ow.ly/LiUQ7
http://ow.ly/LiUMG
http://ow.ly/LiCJS
http://ow.ly/LiCDd
http://ow.ly/LiCwz
http://ow.ly/LiCs4
http://ow.ly/LiCmF
http://ow.ly/LiCiC
http://ow.ly/LiCeE
http://ow.ly/LigQS
http://ow.ly/LigiQ
http://ow.ly/Lig2W
http://ow.ly/LidGT
http://ow.ly/LidDJ
http://ow.ly/LidzD
http://ow.ly/LiduS
http://ow.ly/Lidmt
http://ow.ly/LicX5
http://ow.ly/LicMo
http://ow.ly/LicJy
http://ow.ly/Lghqn
http://ow.ly/Lghiy
http://ow.ly/LghdN
http://ow.ly/LfQ4x
http://ow.ly/LfNTK
http://ow.ly

http://ow.ly/GSLeo
http://ow.ly/GSAxu
http://ow.ly/GSAo5
http://ow.ly/GSA83
http://ow.ly/GSzXI
http://ow.ly/GSzvY
http://ow.ly/GSzmy
http://pbs.twimg.com/media/B6sIaX7CAAAReD0.jpg
http://ow.ly/GSiUs
http://ow.ly/GSiKu
http://ow.ly/GSiCR
http://ow.ly/GSitD
http://ow.ly/GSijs
http://ow.ly/GSi8h
http://ow.ly/GSi1o
http://ow.ly/GShOX
http://ow.ly/GShIM
http://ow.ly/GSfZv
http://ow.ly/GSfRo
http://ow.ly/GSfFK
http://ow.ly/GSeQF
http://ow.ly/GPJbP
http://ow.ly/GPJ5n
http://ow.ly/GPILn
http://ow.ly/GPrhu
http://ow.ly/GPkR8
http://ow.ly/GOM7y
http://ow.ly/GOJN3
http://ow.ly/GOJEV
http://ow.ly/GOHkA
http://ow.ly/GODcL
http://ow.ly/GOCYS
http://ow.ly/GOCKO
http://ow.ly/GOCCX
http://ow.ly/GOukr
http://ow.ly/GOubw
http://ow.ly/GOtBe
http://ow.ly/GOtp2
http://ow.ly/GOtgr
http://ow.ly/GOt6a
http://ow.ly/GOsKx
http://ow.ly/GOsCu
http://ow.ly/GIrUk
http://ow.ly/GIkzT
http://ow.ly/GI7AM
http://ow.ly/GI7wI
http://ow.ly/GHY7B
http://ow.ly/GHXZA
http://ow.ly/GHXU4
http://ow.ly/GHMIG
http://ow.ly/GHLyJ
htt

http://ow.ly/ICHNb
http://ow.ly/ICx9B
http://ow.ly/ICx0N
http://ow.ly/ICvOm
http://ow.ly/ICvVR
http://ow.ly/ICvC1
http://ow.ly/ICvtf
http://ow.ly/ICKK9
http://ow.ly/ICw4B
http://ow.ly/ICBfu
http://ow.ly/ICvhw
http://ow.ly/ICkIv
http://ow.ly/ICkCb
http://ow.ly/Izc5P
http://ow.ly/IzbVF
http://ow.ly/Iz0fK
http://ow.ly/IyVm8
http://ow.ly/IyVbt
http://ow.ly/IyV8X
http://ow.ly/IyAU9
http://ow.ly/IyAMc
http://ow.ly/IyAF7
http://ow.ly/IyA8o
http://ow.ly/IyzZh
http://ow.ly/IyzyG
http://ow.ly/Iyzqa
http://ow.ly/Iyzjv
http://ow.ly/Iyzd5
http://ow.ly/Iyz4w
http://ow.ly/IyyYd
http://ow.ly/IyyFM
http://ow.ly/IyytU
http://ow.ly/Iyyla
http://ow.ly/Iyy2M
http://ow.ly/IyxXq
http://ow.ly/IyxQd
http://ow.ly/IyxeO
http://ow.ly/Iyx7Y
http://ow.ly/IvBEH
http://ow.ly/IuSam
http://ow.ly/IuRUA
http://ow.ly/IumXa
http://ow.ly/Iub0O
http://ow.ly/IuaPJ
http://ow.ly/Iu8XI
http://ow.ly/Iu8R2
http://ow.ly/Iu8Hd
http://ow.ly/Iu8xW
http://ow.ly/Iu8nU
http://ow.ly/Iu8f4
http://ow.ly/Iu87l
http://ow.ly/Iu80H
http://ow.ly

,tweetid,date,title,url
0,585942799561928704,Wed Apr 08 23:10:24 +0000 2015,Injury prevention programs unpopular with high...,http://ow.ly/Lma9z
1,585934004089683969,Wed Apr 08 22:35:27 +0000 2015,6 dietary changes to make midlife,http://ow.ly/LlYfl
2,585926494054195201,Wed Apr 08 22:05:36 +0000 2015,Massachusetts governor gets head shaved to sup...,http://ow.ly/LlLI3
3,585921515922788354,Wed Apr 08 21:45:49 +0000 2015,Dad wins 3 marathons in 8 days; winnings to he...,http://ow.ly/LlLyZ
4,585915150449934338,Wed Apr 08 21:20:32 +0000 2015,Possible cure for melanoma?,http://ow.ly/LlLg8


In [6]:
frame_urls = frame['url'].values

In [ ]:
# Run for this jupyter notebook
title_attempt = True
for key,url in enumerate(frame_urls):

    #     url = 'http://bit.ly/VymaIc'

    # Start the connection
    print(url)
    res = requests.get(url)
    
    # Check status header
    if res.status_code == 200:
        print("Header: ", 200)

        # Initialize BeautifulSoup()
        soup = BeautifulSoup(res.content, 'lxml')

        # Get return content as text
        # text_html = res.text
        
        # Add columns to dataframe
        if title_attempt:
            df_temp = pd.DataFrame(columns=['author','content','tags'])
            frame = pd.concat([frame,df_temp])
            title_attempt = False
        
        ahref = soup.find('a',{'class':'article_link'})
        if ahref:
            author = ahref.text            
#             frame['author'][key] = author[:author.index(",")].lstrip()
            frame['author'][key] = author.lstrip()
        else:
            frame['author'] = ''
        frame['content'][key] = soup.find('div',{'class':'article-body'})
        frame['tags'][key] = get_hastags_out(regex_hastags,frame['title'][key])
        frame['title'][key] = stript_hastags(frame['title'][key])
    else:
        print("[ERROR]: ", res.status_code)

http://ow.ly/Lma9z
[ERROR]:  404
http://ow.ly/LlYfl
[ERROR]:  404
http://ow.ly/LlLI3
[ERROR]:  404
http://ow.ly/LlLyZ
[ERROR]:  404
http://ow.ly/LlLg8
[ERROR]:  404
http://ow.ly/LlL69
[ERROR]:  404
http://ow.ly/LlKYU
[ERROR]:  404
http://ow.ly/LlmOO
[ERROR]:  404
http://ow.ly/LlmKx
[ERROR]:  404
http://ow.ly/Llmaj
[ERROR]:  404
http://ow.ly/LllIf
[ERROR]:  404
http://ow.ly/LllF0
[ERROR]:  404
http://ow.ly/LllBM
[ERROR]:  404
http://ow.ly/Lllze
[ERROR]:  404
http://ow.ly/LllvX
[ERROR]:  404
http://ow.ly/LlRmW
[ERROR]:  404
http://ow.ly/LllsI
[ERROR]:  404
http://ow.ly/LllpI
[ERROR]:  404
http://ow.ly/LlllW
[ERROR]:  404
http://ow.ly/LllhY
[ERROR]:  404
http://ow.ly/Lllah
[ERROR]:  404
http://ow.ly/LlkWI
[ERROR]:  404
http://ow.ly/Llc4n
[ERROR]:  404
http://ow.ly/Ljqd1
[ERROR]:  404
http://ow.ly/LjeGL
[ERROR]:  404
http://ow.ly/LiUWe
[ERROR]:  404
http://ow.ly/LiUSJ
[ERROR]:  404
http://ow.ly/LiUQ7
[ERROR]:  404
http://ow.ly/LiUMG
[ERROR]:  404
http://ow.ly/LiCJS
[ERROR]:  404
http://ow.

scripts.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  
scripts.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
scripts.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Strip "#" within string
scripts.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  def s

http://ow.ly/KUOIc
[ERROR]:  404
http://ow.ly/KTxKM
[ERROR]:  404
http://ow.ly/KTqiA
[ERROR]:  404
http://ow.ly/KTjYF
[ERROR]:  404
http://ow.ly/KT4rX
[ERROR]:  404
http://ow.ly/KT4lJ
[ERROR]:  404
http://ow.ly/KT2Rs
[ERROR]:  404
http://ow.ly/KSVyW
[ERROR]:  404
http://ow.ly/KSUj0
[ERROR]:  404
http://ow.ly/KSSRi
[ERROR]:  404
http://ow.ly/KSyFR
[ERROR]:  404
http://ow.ly/KSyzY
[ERROR]:  404
http://ow.ly/KSyuc
[ERROR]:  404
http://ow.ly/KSymM
[ERROR]:  404
http://ow.ly/KSyim
[ERROR]:  404
http://ow.ly/KSycI
[ERROR]:  404
http://ow.ly/KSPBf
[ERROR]:  404
http://ow.ly/KSy9A
[ERROR]:  404
http://ow.ly/KSy5o
[ERROR]:  404
http://ow.ly/KSy2a
[ERROR]:  404
http://ow.ly/KSxZV
[ERROR]:  404
http://ow.ly/KQAa5
[ERROR]:  404
http://ow.ly/KQlxb
[ERROR]:  404
http://ow.ly/KQ7H1
[ERROR]:  404
http://ow.ly/KQ5KO
[ERROR]:  404
http://ow.ly/KQ5yN
[ERROR]:  404
http://ow.ly/KPDVA
[ERROR]:  404
http://ow.ly/KPDSP
[ERROR]:  404
http://ow.ly/KPDQ2
[ERROR]:  404
http://ow.ly/KPDLK
[ERROR]:  404
http://ow.

In [ ]:
frame.head()

In [ ]:
frame.shape

In [6]:
# If End of jupyter-notebook, reset the "frame" Dataframe as 4 columns
### Writing content into text file
### ================================
file_names = frame['date'].values

for key, name in enumerate(file_names):
    
    # Construct the file name
    located_file = './date/date.txt'
    
    try:
    
        # Writing on file
        contents = frame['date'].iloc[key]
        print(key, ' : ' ,len(contents))
        if len(contents) > 10:
            # Open file for writing
            f= open(located_file,"a")
            
            # Start writing content to file.
            f.write(contents)
            
            # close file connection
            f.close()
        else:
            print("[Out] key: ", key) 
    except:
        pass

### ================================

0  :  30
1  :  30
2  :  30
3  :  30
4  :  30
5  :  30
6  :  30
7  :  30
8  :  30
9  :  30
10  :  30
11  :  30
12  :  30
13  :  30
14  :  30
15  :  30
16  :  30
17  :  30
18  :  30
19  :  30
20  :  30
21  :  30
22  :  30
23  :  30
24  :  30
25  :  30
26  :  30
27  :  30
28  :  30
29  :  30
30  :  30
31  :  30
32  :  30
33  :  30
34  :  30
35  :  30
36  :  30
37  :  30
38  :  30
39  :  30
40  :  30
41  :  30
42  :  30
43  :  30
44  :  30
45  :  30
46  :  30
47  :  30
48  :  30
49  :  30
50  :  30
51  :  30
52  :  30
53  :  30
54  :  30
55  :  30
56  :  30
57  :  30
58  :  30
59  :  30
60  :  30
61  :  30
62  :  30
63  :  30
64  :  30
65  :  30
66  :  30
67  :  30
68  :  30
69  :  30
70  :  30
71  :  30
72  :  30
73  :  30
74  :  30
75  :  30
76  :  30
77  :  30
78  :  30
79  :  30
80  :  30
81  :  30
82  :  30
83  :  30
84  :  30
85  :  30
86  :  30
87  :  30
88  :  30
89  :  30
90  :  30
91  :  30
92  :  30
93  :  30
94  :  30
95  :  30
96  :  30
97  :  30
98  :  30
99  :  30
100  :  30

1062  :  30
1063  :  30
1064  :  30
1065  :  30
1066  :  30
1067  :  30
1068  :  30
1069  :  30
1070  :  30
1071  :  30
1072  :  30
1073  :  30
1074  :  30
1075  :  30
1076  :  30
1077  :  30
1078  :  30
1079  :  30
1080  :  30
1081  :  30
1082  :  30
1083  :  30
1084  :  30
1085  :  30
1086  :  30
1087  :  30
1088  :  30
1089  :  30
1090  :  30
1091  :  30
1092  :  30
1093  :  30
1094  :  30
1095  :  30
1096  :  30
1097  :  30
1098  :  30
1099  :  30
1100  :  30
1101  :  30
1102  :  30
1103  :  30
1104  :  30
1105  :  30
1106  :  30
1107  :  30
1108  :  30
1109  :  30
1110  :  30
1111  :  30
1112  :  30
1113  :  30
1114  :  30
1115  :  30
1116  :  30
1117  :  30
1118  :  30
1119  :  30
1120  :  30
1121  :  30
1122  :  30
1123  :  30
1124  :  30
1125  :  30
1126  :  30
1127  :  30
1128  :  30
1129  :  30
1130  :  30
1131  :  30
1132  :  30
1133  :  30
1134  :  30
1135  :  30
1136  :  30
1137  :  30
1138  :  30
1139  :  30
1140  :  30
1141  :  30
1142  :  30
1143  :  30
1144  :  30
1145

1746  :  30
1747  :  30
1748  :  30
1749  :  30
1750  :  30
1751  :  30
1752  :  30
1753  :  30
1754  :  30
1755  :  30
1756  :  30
1757  :  30
1758  :  30
1759  :  30
1760  :  30
1761  :  30
1762  :  30
1763  :  30
1764  :  30
1765  :  30
1766  :  30
1767  :  30
1768  :  30
1769  :  30
1770  :  30
1771  :  30
1772  :  30
1773  :  30
1774  :  30
1775  :  30
1776  :  30
1777  :  30
1778  :  30
1779  :  30
1780  :  30
1781  :  30
1782  :  30
1783  :  30
1784  :  30
1785  :  30
1786  :  30
1787  :  30
1788  :  30
1789  :  30
1790  :  30
1791  :  30
1792  :  30
1793  :  30
1794  :  30
1795  :  30
1796  :  30
1797  :  30
1798  :  30
1799  :  30
1800  :  30
1801  :  30
1802  :  30
1803  :  30
1804  :  30
1805  :  30
1806  :  30
1807  :  30
1808  :  30
1809  :  30
1810  :  30
1811  :  30
1812  :  30
1813  :  30
1814  :  30
1815  :  30
1816  :  30
1817  :  30
1818  :  30
1819  :  30
1820  :  30
1821  :  30
1822  :  30
1823  :  30
1824  :  30
1825  :  30
1826  :  30
1827  :  30
1828  :  30
1829